In [6]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

train_df = pd.read_csv("./data/train_processed4.csv")
test_df = pd.read_csv("./data/test_processed4.csv")
test_df = test_df.drop(['cc_num'], axis=1)
# scaled_cols = ['city_pop']
# # Initialize the StandardScaler
# scaler = StandardScaler()
# # Fit on the training set and transform both training and validation sets
# train_df[scaled_cols] = scaler.fit_transform(train_df[scaled_cols])

# Assuming 'train_df' includes both features and the target ('is_fraud')
X = train_df.drop(['is_fraud', 'Id', 'cc_num'], axis=1)  # Dropping the target to isolate features
y = train_df['is_fraud']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [7]:
print(X_train.head())

        merchant  category    amt  gender  city  state  city_pop  job  \
303198       398         7  48.31       1   233     11      1195  463   
204087       476         0  46.61       0    98     33   2504700  279   
196192       371        10  65.22       0   437     16    736284  138   
101664       671         7  83.38       1   723     21     75830  148   
251609        61         7  53.89       0   454     49       241  139   

        Time_Delta  distance_to_prev  location_consistency  hour  day_of_week  \
303198       113.0         56.967992              2.108487    14            1   
204087       198.0         51.650475              2.172542    22            5   
196192       270.0         12.022653              2.088555    22            6   
101664       229.0         35.709673              2.166446    21            6   
251609       411.0         57.039024              2.124517    12            1   

        age  dist_to_home  amt_relative_avg  
303198   46     46.868069   

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
# Initialize a basic Random Forest model
rf = RandomForestClassifier(criterion='entropy', max_depth=None, min_samples_leaf=4, min_samples_split=10, n_estimators=200, class_weight='balanced_subsample', random_state=40)
#rf = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42)

rf.fit(X_train, y_train)

# Prediction and Evaluation
y_pred_rf = rf.predict(X_val)
print(f1_score(y_val, y_pred_rf))
print(classification_report(y_val, y_pred_rf))



0.7957317073170732
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     96876
         1.0       0.93      0.70      0.80       375

    accuracy                           1.00     97251
   macro avg       0.96      0.85      0.90     97251
weighted avg       1.00      1.00      1.00     97251



In [9]:
from sklearn.ensemble import BaggingClassifier

bag_clf = BaggingClassifier(rf, n_estimators=200, n_jobs=-1, random_state=42).fit(X_train, y_train)

y_pred_bag = bag_clf.predict(X_val)

val_f1 = f1_score(y_val, y_pred_bag)

print(val_f1)
print(classification_report(y_val, y_pred_bag))

0.7723076923076924
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     96876
         1.0       0.91      0.67      0.77       375

    accuracy                           1.00     97251
   macro avg       0.96      0.83      0.89     97251
weighted avg       1.00      1.00      1.00     97251



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import pickle

# Decision Tree Classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Parameters for GridSearchCV
param_grid = {
    'max_depth': [7, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [2, 4, 8],
    'criterion': ['gini', 'entropy']
}

# GridSearchCV
grid_search = GridSearchCV(dt_classifier, param_grid, cv=5, scoring='f1_micro', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Best parameters and score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Validation
best_dt = grid_search.best_estimator_

with open('dt_model4.obj', 'wb') as f:
        pickle.dump(best_dt, f)


y_pred_dt = best_dt.predict(X_val)

val_f1 = f1_score(y_val, y_pred_dt)

print(best_params, best_score, val_f1)
print(classification_report(y_val, y_pred_dt))

In [3]:
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

param_grid_knn = {
    'n_neighbors': range(1, 21),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

knn = KNeighborsClassifier()

grid_search_knn = GridSearchCV(knn, param_grid_knn, cv=5, scoring='f1_micro', n_jobs=-1, verbose=3)

grid_search_knn.fit(X_train, y_train)

best_params_knn = grid_search_knn.best_params_
best_score_knn = grid_search_knn.best_score_

best_knn = grid_search_knn.best_estimator_
# with open('best_knn.obj', 'wb') as f:
#     pickle.dump(best_knn, f)
y_pred_knn = best_knn.predict(X_val)
f1_score_knn = f1_score(y_val, y_pred_knn)
print(best_params_knn, best_score_knn, f1_score_knn)

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[CV 1/5] END metric=euclidean, n_neighbors=2, weights=distance;, score=0.995 total time= 2.3min
[CV 2/5] END metric=euclidean, n_neighbors=1, weights=distance;, score=0.995 total time= 2.3min
[CV 3/5] END metric=euclidean, n_neighbors=1, weights=distance;, score=0.995 total time= 2.3min
[CV 5/5] END metric=euclidean, n_neighbors=1, weights=distance;, score=0.995 total time= 2.4min
[CV 1/5] END metric=euclidean, n_neighbors=1, weights=distance;, score=0.995 total time= 2.4min
[CV 4/5] END metric=euclidean, n_neighbors=1, weights=distance;, score=0.995 total time= 2.4min
[CV 2/5] END metric=euclidean, n_neighbors=1, weights=uniform;, score=0.995 total time= 2.4min
[CV 3/5] END metric=euclidean, n_neighbors=1, weights=uniform;, score=0.995 total time= 2.4min
[CV 5/5] END metric=euclidean, n_neighbors=1, weights=uniform;, score=0.995 total time= 2.4min
[CV 3/5] END metric=euclidean, n_neighbors=2, weights=uniform;, score=0.996 t

KeyboardInterrupt: 